<a href="https://colab.research.google.com/github/daa103/DAA_ESAA/blob/main/SSL_self_training_LightGBM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import random

# os 관리를 위한 라이브러리
import os
import gc

In [2]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)

In [3]:
seed_everything(42) # Seed 고정

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
### 전처리된 파일 불러오기

train = pd.read_parquet('/content/drive/MyDrive/ESAA_data/train_pre.parquet')
test = pd.read_parquet('/content/drive/MyDrive/ESAA_data/test_pre.parquet')
sample_submission = pd.read_csv('/content/drive/MyDrive/ESAA_data/sample_submission.csv', index_col = 0)

In [5]:
train.head(3)

,Month,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_Airport_ID,Destination_Airport_ID,Distance,Carrier_ID(DOT),Tail_Number,Delay,Holiday
1,8,460,624,13930,14869,1250.0,20304,310,NaN,0
4,1,540,619,14771,10157,250.0,20304,554,NaN,0
5,4,945,1011,11618,11278,199.0,20452,3430,1.0,0


In [6]:
test.head(3)

,Month,Estimated_Departure_Time,Estimated_Arrival_Time,Origin_Airport_ID,Destination_Airport_ID,Distance,Carrier_ID(DOT),Tail_Number,Holiday
0,12,716,779,12266,14683,191.0,19977,4382,0
1,9,900,1035,11618,10397,746.0,19790,1934,0
2,3,960,1155,13930,12953,733.0,19977,2144,0


# **3. 준지도학습**
- 라벨이 없는 데이터에 대한 라벨링 진행

## **3-0. 데이터 분리**

In [7]:
### 데이터 분리
# 레이블이 있는 데이터와 없는 데이터 분리

train_labeled = train.loc[train['Delay'].notnull(),:]
train_unlabeled = train.loc[train['Delay'].isnull(),:]

In [8]:
train_labeled.shape[0]

172245

In [9]:
train_unlabeled.shape[0]

502770

- 라벨링 된 데이터에 비해 라벨링 되지 **않은** 데이터의 수가 더 많다.

In [10]:
### X,y 분리

# train
X_labeled = train_labeled.drop(['Delay'],axis = 1)
y_labeled = train_labeled['Delay']
X_unlabeled = train_unlabeled.drop(['Delay'],axis = 1)

print("X Labeled: ", X_labeled.shape)
print("y Labeled: ", y_labeled.shape)
print()
print("X Unlabeled: ", X_unlabeled.shape)

# test
X_test = test
print()
print("X Test: ", X_test.shape)

X Labeled:  (172245, 9)
y Labeled:  (172245,)

X Unlabeled:  (502770, 9)

X Test:  (1000000, 9)


## **3-1. 기본 모델 모델링(Base Model Modeling)**
- Base Model 선정 & 학습
  - 레이블이 **있는** 데이터만을 사용하여 모델을 학습시키면서 튜닝을 진행
- 기본 분류기인 Base Model(ex> LogisticRegression)의 하이퍼 파라미터 튜닝을 수행
  - GridSearchCV, RandomizedSearchCV, 또는 Bayesian optimization 등의 방법 활용
  - 주어진 데이터와 문제에 맞는 하이퍼파라미터 조합을 찾는 것이 중요

In [ ]:
### 여러 모델을 모델링 해보고 성능 평가
# 일반적인 분류 모델링과 동일
# 각 모델의 성능은 교차 검증을 통해 측정할 예정
# 이때는 X_labeled, y_labeled를 활용

### 하이퍼 파라미터 튜닝
# 가장 성능이 좋은 모델에 대해 하이퍼 파라미터 수행

In [11]:
from sklearn.semi_supervised import SelfTrainingClassifier
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from lightgbm import LGBMClassifier
import numpy as np
from sklearn.datasets import make_classification
from lightgbm import LGBMClassifier

# 기본 분류기도 LightGBM 설정
base_model = LGBMClassifier()

In [ ]:
# 일단은 가장 기본적인 모델로 진행

#from sklearn.linear_model import LogisticRegression
#base_model = LogisticRegression() # base model로 활용할 모델 객체 생성

# 이때는 다시 base model을 학습시킬 필요는 없음
# 밑에서 SelfTrainingClassifier 객체를 생성하고 해당 모델을 학습시킴

## **3-2. Self Training**
- 초기에 레이블이 없는 데이터를 레이블링하여 일부 레이블이 있는 데이터를 생성한 후, 이 데이터를 활용해 모델을 학습시키는 방법
- 학습된 모델을 사용하여 레이블이 없는 데이터에 대한 예측을 수행하고, 예측 결과 중 신뢰할 수 있는 일부 데이터를 레이블링에 추가로 사용하여 새로운 학습용 데이터를 생성
> 해당 과정을 반복하여 모델을 업데이트하면서 점진적으로 더 많은 레이블을 확보해나가는 방식

In [12]:
# Self Training 분류기 생성
self_training_model = SelfTrainingClassifier(base_model, criterion='k_best', k_best=100)

In [13]:
### Self Training을 위한 함수

def run_self_training(self_training_model, X_labeled, y_labeled,
                      X_unlabeled, early_stopping_rounds = 5):
  iter = 0 # 반복 횟수 카운트를 위한 변수
  rounds_without_improvement = 0 # early stopping을 위한 카운트 변수

  while X_unlabeled.shape[0] > 0 and rounds_without_improvement < early_stopping_rounds:
    iter = iter + 1
    print("=== Iteration {} ===".format(iter))

    ## 1. 레이블이 있는(labeled) 데이터로 모델 학습
    self_training_model.fit(X_labeled, y_labeled)

    ## 2. 훈련된 모델로 레이블이 없는 데이터를 분류
    y_pred_unlabeled = self_training_model.predict(X_unlabeled)

    ## 3. 의사 레이블(pseudo label) 설정
    # 모델이 90% 이상 확신하는 데이터를 labeled 데이터에 추가
    confident_mask = self_training_model.predict_proba(X_unlabeled).max(axis = 1) > 0.9
    X_confident = X_unlabeled[confident_mask]
    y_confident = y_pred_unlabeled[confident_mask]
    # 신뢰할 수 있는 데이터를 레이블링에 추가하여 새로운 학습용 데이터 구성
    # 레이블링 된 데이터의 수를 증가시키는 과정
    X_labeled = np.concatenate([X_labeled, X_confident], axis = 0)
    y_labeled = np.concatenate([y_labeled, y_confident], axis = 0)
    # 의사 레이블링 된 데이터들은 이제 레이블이 없는 데이터에서 제거
    X_unlabeled = X_unlabeled[~confident_mask]

    print("{}개 레이블링 완료".format(X_confident.shape[0]))

    ## Early Stopping 체크
    if X_confident.shape[0] <= 0:
      rounds_without_improvement += 1
      print("Early Stopping 카운트: {}/{}".format(rounds_without_improvement,early_stopping_rounds))
      print()
    else:
      rounds_without_improvement = 0
      print()

  print()
  print("레이블링 완료")

  # 라벨링 된 데이터 반환
  return X_labeled, y_labeled, X_unlabeled

In [17]:
# Self-training을 통한 레이블링 수행

X_labeled, y_labeled, X_unlabeled = run_self_training(base_model, X_labeled, y_labeled, X_unlabeled, early_stopping_rounds = 10)

=== Iteration 1 ===
70702개 레이블링 완료

=== Iteration 2 ===
47252개 레이블링 완료

=== Iteration 3 ===
36148개 레이블링 완료

=== Iteration 4 ===
27143개 레이블링 완료

=== Iteration 5 ===
19174개 레이블링 완료

=== Iteration 6 ===
14161개 레이블링 완료

=== Iteration 7 ===
11367개 레이블링 완료

=== Iteration 8 ===
11432개 레이블링 완료

=== Iteration 9 ===
10441개 레이블링 완료

=== Iteration 10 ===
8895개 레이블링 완료

=== Iteration 11 ===
5650개 레이블링 완료

=== Iteration 12 ===
3903개 레이블링 완료

=== Iteration 13 ===
4825개 레이블링 완료

=== Iteration 14 ===
6605개 레이블링 완료

=== Iteration 15 ===
7240개 레이블링 완료

=== Iteration 16 ===
7083개 레이블링 완료

=== Iteration 17 ===
6068개 레이블링 완료

=== Iteration 18 ===
4691개 레이블링 완료

=== Iteration 19 ===
3336개 레이블링 완료

=== Iteration 20 ===
2624개 레이블링 완료

=== Iteration 21 ===
1323개 레이블링 완료

=== Iteration 22 ===
1272개 레이블링 완료

=== Iteration 23 ===
1568개 레이블링 완료

=== Iteration 24 ===
1311개 레이블링 완료

=== Iteration 25 ===
1534개 레이블링 완료

=== Iteration 26 ===
1618개 레이블링 완료

=== Iteration 27 ===
1910개 레이블링 완료

=== Iteration 28 ===
2750개 레

In [18]:
print(X_labeled.shape[0])
print(X_unlabeled.shape[0]/(X_labeled.shape[0] + X_unlabeled.shape[0]))

538004
0.20297474870928794


- 라벨링 된 데이터의 개수가 172245에서 538004로 증가
- 아직 약 20%의 데이터가 라벨링되지 않은 상황

In [19]:
### 최종 데이터 병합- 20%데이터는 무시

X_train_final = X_labeled
y_train_final = y_labeled

# **4. 최종 예측을 위한 모델링**

In [ ]:
### 여러 모델을 모델링 해보고 성능 평가
# 일반적인 분류 모델링과 동일
# 각 모델의 성능은 교차 검증을 통해 측정할 예정
# 이때는 X_train_final, y_train_final을 활용

### 하이퍼 파라미터 튜닝
# 가장 성능이 좋은 모델에 대해 하이퍼 파라미터 수행

In [20]:
from lightgbm import LGBMClassifier

final_classifier = LGBMClassifier(n_estimators=500)

final_classifier.fit(X_train_final, y_train_final)
y_test_pred = final_classifier.predict_proba(X_test)

In [21]:
y_test_pred

array([[0.03049928, 0.96950072],
       [0.0688094 , 0.9311906 ],
       [0.31183463, 0.68816537],
       ...,
       [0.29225355, 0.70774645],
       [0.12828192, 0.87171808],
       [0.03900285, 0.96099715]])

- Delayed = 0
- Not_Delayed = 1

In [22]:
sample_submission.head(3)

,Not_Delayed,Delayed
ID,,
TEST_000000,0,1
TEST_000001,0,1
TEST_000002,0,1


In [23]:
# 현재 컬럼 순서가 반대이므로 바꿔주기

y_test_pred = y_test_pred[:, [1, 0]]

In [24]:
submission = pd.DataFrame(data = y_test_pred, columns = sample_submission.columns,
                          index = sample_submission.index)

In [25]:
submission.head(3)

,Not_Delayed,Delayed
ID,,
TEST_000000,0.969501,0.030499
TEST_000001,0.931191,0.068809
TEST_000002,0.688165,0.311835


In [26]:
submission.to_csv('baseline_submission.csv', index = True)

**GridSearchCV를 활용하여 하이퍼파라미터 튜닝 적용**